In [1]:
# -----------------------------------------------------------------------------
# Authors:      Rafael Ballester-Ripoll <rballester@ifi.uzh.ch>
#
# Copyright:    ttrecipes project (c) 2017-2018
#               VMMLab - University of Zurich
#
# ttrecipes is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published
# by the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# ttrecipes is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with ttrecipes.  If not, see <http://www.gnu.org/licenses/>.
# -----------------------------------------------------------------------------

from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
from unittest import TestCase
import numpy as np
# import tt
import teneva
import sampling
import completion
import util
import sparse
from util import nr

# import ttrecipes as tr


class TestCompletion(TestCase):

    def test_categorical(self):
        N = 4
        Is = [5, ]*N
        ranks = 3
        gt = teneva.rand(Is, ranks)
        P = int(np.prod(nr(gt)[0])/10)
#         Xs = tr.core.LHS(Is, P)
        Xs = sampling.LHS(Is, P)
#         ys = np.array([gt[x] for x in Xs])
        ys = np.array([util.getitem(gt, x) for x in Xs])
#         completed = tr.core.categorical_ALS(Xs, ys, shape=Is, ranks=ranks, verbose=False)
        completed = completion.categorical_ALS(Xs, ys, shape=Is, ranks=ranks, verbose=False)
#         reco = np.array([completed[x] for x in Xs])
        reco = np.array([util.getitem(completed, x) for x in Xs])
        self.assertLessEqual(np.linalg.norm(reco - ys)/np.linalg.norm(ys), 0.25)

    def test_continuous(self):
        N = 4
        Is = [15, ]*N
        Rs = [1] + [4, ]*(N-1) + [1]
        Ss = [3, ]*N
#         cores = tt.vector.to_list(tr.core.random_tt(Ss, Rs))
        cores = (teneva.rand(Ss, Rs))
#         Us = tr.core.generate_bases("legendre", Is, Ss)
        Us = util.generate_bases("legendre", Is, Ss)
        cores = [np.einsum('ijk,lj->ilk', c, U) for c, U in zip(cores, Us)]
#         gt = tt.vector.from_list(cores)
        gt = cores
        n, r = nr(gt)
        P = int(np.prod(Is) / 10)
#         Xs = tr.core.LHS(gt.n, P)
        Xs = sampling.LHS(n, P)
#         ys = tr.core.sparse_reco(gt, Xs)
        ys = sparse.sparse_reco(gt, Xs)
        completed = completion.pce_interpolation(Xs, ys, shape=Is, ranks=Rs, ranks2=Ss, maxswp=10, verbose=True)
#         reco = tr.core.sparse_reco(completed, Xs)
        reco = sparse.sparse_reco(completed, Xs)
        self.assertLessEqual(np.linalg.norm(reco - ys) / np.linalg.norm(ys), 1e-5)


In [2]:
TestCompletion().test_categorical()

In [3]:
TestCompletion().test_continuous()

Interpolating a 4D tensor of size [15, 15, 15, 15] using 5062 samples and ranks [1, 4, 4, 4, 1]...
Sweep:   0 | eps: 0.0230269253200630 | time: 0.1350
Sweep:   1 | eps: 0.0006300873187343 | time: 0.1450
Sweep:   2 | eps: 0.0000223140308900 | time: 0.1700
Sweep:   3 | eps: 0.0000049662602034 | time: 0.1555
Sweep:   4 | eps: 0.0000036863401719 | time: 0.1200
Sweep:   5 | eps: 0.0000031185526316 | time: 0.1140
Sweep:   6 | eps: 0.0000043258882567 | time: 0.1190
Sweep:   7 | eps: 0.0000037082949237 | time: 0.1240
Sweep:   8 | eps: 0.0000022414979664 | time: 0.1170
Sweep:   9 | eps: 0.0000028162301589 | time: 0.1240
